In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

CNNModel = Sequential()

CNNModel.add(Conv2D(64, (9, 9), input_shape=(100,100,3),padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Activation('relu'))
CNNModel.add(MaxPooling2D(pool_size=(2, 2)))

CNNModel.add(Conv2D(128, (5, 5), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Activation('relu'))
CNNModel.add(MaxPooling2D(pool_size=(2, 2)))

CNNModel.add(ZeroPadding2D((1, 1)))
CNNModel.add(Conv2D(256, (3, 3), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Activation('relu'))
CNNModel.add(MaxPooling2D(pool_size=(2, 2)))

CNNModel.add(ZeroPadding2D((1, 1)))
CNNModel.add(Conv2D(512, (3, 3), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Activation('relu'))

CNNModel.add(ZeroPadding2D((1, 1)))
CNNModel.add(Conv2D(512, (3, 3), padding='same'))
CNNModel.add(BatchNormalization())
CNNModel.add(Activation('relu'))
CNNModel.add(MaxPooling2D(pool_size=(2, 2)))

CNNModel.add(Flatten())
CNNModel.add(Dense(64))
CNNModel.add(BatchNormalization())
CNNModel.add(Activation('relu'))
CNNModel.add(Dropout(0.5))

CNNModel.add(Dense(128))
CNNModel.add(BatchNormalization())
CNNModel.add(Activation('relu'))
CNNModel.add(Dropout(0.5))

CNNModel.add(Dense(1))
CNNModel.add(BatchNormalization())
CNNModel.add(Activation('sigmoid'))

CNNModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=30,shear_range=0.2, zoom_range=[0.8, 1.2],horizontal_flip=True,fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('E:/RIGA_200_Images',target_size = (100, 100),batch_size = 64, class_mode = 'binary')

test_set = test_datagen.flow_from_directory('E:/RIGA_200_Images',target_size = (100, 100), batch_size = 64,class_mode = 'binary')

my_callbacks = [ tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1), tf.keras.callbacks.ModelCheckpoint('cnn_model.h5',verbose=1, save_best_only=True, save_weights_only=False) 
    ]

CNNModel.fit(training_set, epochs=200, validation_data = test_set, callbacks=my_callbacks)
CNNModel.save('cnn_model.h5')


In [ ]:
# Stream lit for making the ui of the program
import streamlit as st
import tensorflow as tf
from PIL import Image, ImageOps
import numpy as np

st.set_option('deprecation.showfileUploaderEncoding', False)
# @st.cache(suppress_st_warning=True,allow_output_mutation=True)
def import_and_predict(image_data, model):
    image = ImageOps.fit(image_data, (100,100),Image.ANTIALIAS)
    image = image.convert('RGB')
    image = np.asarray(image)
    st.image(image, channels='RGB')
    image = (image.astype(np.float32) / 255.0)
    img_reshape = image[np.newaxis,...]
    prediction = model.predict(img_reshape)
    return prediction
model = tf.keras.models.load_model('cnn_model.h5')

st.write("""# ***Glaucoma detection ***""")

st.write("This is a simple image classification web app to predict glaucoma through fundus image of eye")

file = st.file_uploader("Please upload an image(jpg) file", type=["jpg"])

if file is None:
    st.text("You haven't uploaded a jpg image file")
else:
    imageI = Image.open(file)
    prediction = import_and_predict(imageI, model)
    pred = prediction[0][0]
    if(pred > 0.5):
        st.write(""" ## **Prediction:** You eye is Healthy. Great!!""")
        st.balloons()
    else:
        st.write("""## **Prediction:** You are affected by Glaucoma! Please consult an ophthalmologist as soon as possible.""")